<a href="https://colab.research.google.com/github/joynaomi81/Olusanya-Joy-Naomi-Undergraduate-Long-Essay-Code/blob/main/facebook_mms_1b_all_(ASR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/yoruba_dataset_with_no_tone.csv")
df.head()

,sentence,full_audio_path,no_tone
0,Ó ti pé ọgọ́rùn-ún ọjọ́ tí ìjọba ti f'òfin de wá.,/content/drive/MyDrive/clips/common_voice_yo_3...,O ti pe ogorun-un ojo ti ijoba ti f'ofin de wa.
1,Gómìnà obìnrin ẹyọ kan péré la ti ní ní ìp...,/content/drive/MyDrive/clips/common_voice_yo_3...,Gomina obinrin eyo kan pere la ti ni ni ipinle...
2,Inú àbótí ńlá ni Àjàní kó gbogbo àwọn aṣọ rẹ̀ sí.,/content/drive/MyDrive/clips/common_voice_yo_3...,Inu aboti nla ni Ajani ko gbogbo awon aso re si.
3,Kò sí ìwé ẹ̀rí fun olùdíje tí àwọn ẹgbẹ́ oni...,/content/drive/MyDrive/clips/common_voice_yo_3...,Ko si iwe eri fun oludije ti awon egbe onigbaa...
4,Àmọ̀tẹ́kùn ti gba àṣẹ ìdáábòbò lọ́wọ́ ...,/content/drive/MyDrive/clips/common_voice_yo_3...,Amotekun ti gba ase idaabobo lowo ijoba.


In [4]:
df.shape

(3099, 3)

In [ ]:
!pip install transformers datasets torchaudio accelerate jiwer wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 110.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

## Fine-Tuning ASR Model with Tone-Marked Yoruba Dataset

In [ ]:
#Import the necessary libraries
import torch
import wandb
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoProcessor, AutoModelForCTC
import jiwer
import numpy as np


# Initialize wandb
wandb.init(
    project="MMS-ASR-tone-yoruba-finetune",
    name="run-1",
    config={
        "model": "facebook/mms-1b-all",
        "language": "yoruba",
        "batch_size": 4,
        "learning_rate": 1e-5,
        "epochs": 3,
    }
)


class AudioDataset(Dataset):
    def __init__(self, df, processor):
        self.data = df.reset_index(drop=True)
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = self.data.loc[idx, "full_audio_path"]
        transcription = self.data.loc[idx, "sentence"]

        try:
            waveform, sr = torchaudio.load(audio_path)
            if sr != 16000:
                waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
            waveform = waveform.squeeze().numpy()
            if waveform.size == 0:
                print(f"Skipping empty audio file at {audio_path}")
                return None

            input_features = self.processor(
                waveform, sampling_rate=16000, return_tensors="pt"
            ).input_values[0]

            if not transcription or pd.isna(transcription):
                print(f"Skipping item with empty transcription for {audio_path}")
                return None

            labels = self.processor.tokenizer(transcription, return_attention_mask=False).input_ids
            if not labels or not isinstance(labels, list):
                print(f"Invalid labels for transcription '{transcription}' at {audio_path}")
                return None

            return {
                "input_features": input_features,
                "labels": torch.tensor(labels, dtype=torch.long),
                "text": transcription,
            }
        except Exception as e:
            print(f"Error processing item {idx} at {audio_path}: {e}")
            return None


# Tone mapping for Yoruba vowels
TONE_MAP = {
    'á': ('a', 'H'), 'à': ('a', 'L'), 'a': ('a', 'M'),
    'é': ('e', 'H'), 'è': ('e', 'L'), 'e': ('e', 'M'),
    'ẹ́': ('ẹ', 'H'), 'ẹ̀': ('ẹ', 'L'), 'ẹ': ('ẹ', 'M'),
    'í': ('i', 'H'), 'ì': ('i', 'L'), 'i': ('i', 'M'),
    'ó': ('o', 'H'), 'ò': ('o', 'L'), 'o': ('o', 'M'),
    'ọ́': ('ọ', 'H'), 'ọ̀': ('ọ', 'L'), 'ọ': ('ọ', 'M'),
    'ú': ('u', 'H'), 'ù': ('u', 'L'), 'u': ('u', 'M')
}

def extract_tone_sequence(text):
    tone_sequence = []
    for char in text:
        if char in TONE_MAP:
            _, tone = TONE_MAP[char]
            tone_sequence.append(tone)
    return tone_sequence

def compute_levenshtein(ref, hyp):
    dp = np.zeros((len(ref) + 1, len(hyp) + 1), dtype=int)
    for i in range(len(ref) + 1):
        dp[i][0] = i
    for j in range(len(hyp) + 1):
        dp[0][j] = j
    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            cost = 0 if ref[i-1] == hyp[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + cost
            )
    return dp[-1][-1]

def tone_error_rate(references, predictions):
    total_tones = 0
    total_errors = 0
    for ref_text, pred_text in zip(references, predictions):
        ref_tones = extract_tone_sequence(ref_text)
        pred_tones = extract_tone_sequence(pred_text)
        total_tones += len(ref_tones)
        total_errors += compute_levenshtein(ref_tones, pred_tones)
    return total_errors / total_tones if total_tones > 0 else 0.0


def evaluate(model, dataloader, processor, device):
    model.eval()
    preds = []
    refs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            if batch is None:
                continue

            inputs = batch["input_features"].to(device)
            labels = batch["labels"].to(device)

            logits = model(inputs).logits
            predicted_ids = torch.argmax(logits, dim=-1)
            transcripts = processor.batch_decode(predicted_ids, skip_special_tokens=True)

            preds.extend(transcripts)
            refs.extend(batch["text"])

    wer_score = jiwer.wer(refs, preds)
    ter_score = tone_error_rate(refs, preds)
    return wer_score, ter_score


# Load data
csv_path = "/content/drive/MyDrive/yoruba_dataset_with_no_tone.csv"
df = pd.read_csv(csv_path)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


# Load model and processor
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("facebook/mms-1b-all")
model = AutoModelForCTC.from_pretrained("facebook/mms-1b-all")

def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if not batch:
        return None

    input_features = [item["input_features"] for item in batch]
    label_features = [item["labels"] for item in batch]
    texts = [item["text"] for item in batch]

    input_features = torch.nn.utils.rnn.pad_sequence(input_features, batch_first=True)

    padded_labels = processor.tokenizer.pad(
        {"input_ids": label_features},
        return_tensors="pt",
        padding="longest"
    )
    label_features = padded_labels["input_ids"]

    return {
        "input_features": input_features,
        "labels": label_features,
        "text": texts,
    }


train_dataset = AudioDataset(train_df, processor)
val_dataset = AudioDataset(val_df, processor)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)


for epoch in range(wandb.config.epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} Training"):
        if batch is None:
            print("Skipping empty batch")
            continue

        inputs = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_values=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        wandb.log({"train/loss": loss.item()})

    avg_train_loss = total_loss / len(train_loader)
    val_wer, val_ter = evaluate(model, val_loader, processor, device)

    wandb.log({
        "epoch": epoch + 1,
        "train/loss_avg": avg_train_loss,
        "val/wer": val_wer,
        "val/ter": val_ter,
    })

    print(f"Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Val WER: {val_wer:.4f} | Val TER: {val_ter:.4f}")




<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: joynaomi81 (joynaomi81-friendnpal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

...

Epoch 1 | Train Loss: 0.3063 | Val WER: 0.8692 | Val TER: 0.4648


Evaluating: 100%|██████████| 155/155 [00:41<00:00,  3.77it/s]


Epoch 2 | Train Loss: 0.1524 | Val WER: 0.8897 | Val TER: 0.5016


Evaluating: 100%|██████████| 155/155 [00:41<00:00,  3.77it/s]


Epoch 3 | Train Loss: 0.0870 | Val WER: 0.8508 | Val TER: 0.5237


## Fine-Tuning ASR Model without Tone-Marked Yoruba Dataset

In [ ]:
#Import necessary libraries
import torch
import wandb
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AutoProcessor, AutoModelForCTC
import jiwer
import numpy as np

# Initialize wandb
wandb.init(
    project="MMS-ASR-tone-yoruba-finetune",
    name="run-1",
    config={
        "model": "facebook/mms-1b-all",
        "language": "yoruba",
        "batch_size": 4,
        "learning_rate": 1e-5,
        "epochs": 3,
    }
)


class AudioDataset(Dataset):
    def __init__(self, df, processor):
        self.data = df.reset_index(drop=True)
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_path = self.data.loc[idx, "full_audio_path"]
        transcription = self.data.loc[idx, "no_tone"]

        try:
            waveform, sr = torchaudio.load(audio_path)
            if sr != 16000:
                waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
            waveform = waveform.squeeze().numpy()
            if waveform.size == 0:
                print(f"Skipping empty audio file at {audio_path}")
                return None

            input_features = self.processor(
                waveform, sampling_rate=16000, return_tensors="pt"
            ).input_values[0]

            if not transcription or pd.isna(transcription):
                print(f"Skipping item with empty transcription for {audio_path}")
                return None

            labels = self.processor.tokenizer(transcription, return_attention_mask=False).input_ids
            if not labels or not isinstance(labels, list):
                print(f"Invalid labels for transcription '{transcription}' at {audio_path}")
                return None

            return {
                "input_features": input_features,
                "labels": torch.tensor(labels, dtype=torch.long),
                "text": transcription,
            }
        except Exception as e:
            print(f"Error processing item {idx} at {audio_path}: {e}")
            return None


# Tone mapping for Yoruba vowels
TONE_MAP = {
    'á': ('a', 'H'), 'à': ('a', 'L'), 'a': ('a', 'M'),
    'é': ('e', 'H'), 'è': ('e', 'L'), 'e': ('e', 'M'),
    'ẹ́': ('ẹ', 'H'), 'ẹ̀': ('ẹ', 'L'), 'ẹ': ('ẹ', 'M'),
    'í': ('i', 'H'), 'ì': ('i', 'L'), 'i': ('i', 'M'),
    'ó': ('o', 'H'), 'ò': ('o', 'L'), 'o': ('o', 'M'),
    'ọ́': ('ọ', 'H'), 'ọ̀': ('ọ', 'L'), 'ọ': ('ọ', 'M'),
    'ú': ('u', 'H'), 'ù': ('u', 'L'), 'u': ('u', 'M')
}

def extract_tone_sequence(text):
    tone_sequence = []
    for char in text:
        if char in TONE_MAP:
            _, tone = TONE_MAP[char]
            tone_sequence.append(tone)
    return tone_sequence

def compute_levenshtein(ref, hyp):
    dp = np.zeros((len(ref) + 1, len(hyp) + 1), dtype=int)
    for i in range(len(ref) + 1):
        dp[i][0] = i
    for j in range(len(hyp) + 1):
        dp[0][j] = j
    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            cost = 0 if ref[i-1] == hyp[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,
                dp[i][j-1] + 1,
                dp[i-1][j-1] + cost
            )
    return dp[-1][-1]

def tone_error_rate(references, predictions):
    total_tones = 0
    total_errors = 0
    for ref_text, pred_text in zip(references, predictions):
        ref_tones = extract_tone_sequence(ref_text)
        pred_tones = extract_tone_sequence(pred_text)
        total_tones += len(ref_tones)
        total_errors += compute_levenshtein(ref_tones, pred_tones)
    return total_errors / total_tones if total_tones > 0 else 0.0


def evaluate(model, dataloader, processor, device):
    model.eval()
    preds = []
    refs = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            if batch is None:
                continue

            inputs = batch["input_features"].to(device)
            labels = batch["labels"].to(device)

            logits = model(inputs).logits
            predicted_ids = torch.argmax(logits, dim=-1)
            transcripts = processor.batch_decode(predicted_ids, skip_special_tokens=True)

            preds.extend(transcripts)
            refs.extend(batch["text"])

    wer_score = jiwer.wer(refs, preds)
    ter_score = tone_error_rate(refs, preds)
    return wer_score, ter_score


# Load data
csv_path = "/content/drive/MyDrive/yoruba_dataset_with_no_tone.csv"
df = pd.read_csv(csv_path)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


# Load model and processor
# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("facebook/mms-1b-all")
model = AutoModelForCTC.from_pretrained("facebook/mms-1b-all")

def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if not batch:
        return None

    input_features = [item["input_features"] for item in batch]
    label_features = [item["labels"] for item in batch]
    texts = [item["text"] for item in batch]

    input_features = torch.nn.utils.rnn.pad_sequence(input_features, batch_first=True)

    padded_labels = processor.tokenizer.pad(
        {"input_ids": label_features},
        return_tensors="pt",
        padding="longest"
    )
    label_features = padded_labels["input_ids"]

    return {
        "input_features": input_features,
        "labels": label_features,
        "text": texts,
    }


train_dataset = AudioDataset(train_df, processor)
val_dataset = AudioDataset(val_df, processor)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)


for epoch in range(wandb.config.epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} Training"):
        if batch is None:
            print("Skipping empty batch")
            continue

        inputs = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_values=inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        wandb.log({"train/loss": loss.item()})

    avg_train_loss = total_loss / len(train_loader)
    val_wer, val_ter = evaluate(model, val_loader, processor, device)

    wandb.log({
        "epoch": epoch + 1,
        "train/loss_avg": avg_train_loss,
        "val/wer": val_wer,
        "val/ter": val_ter,
    })

    print(f"Epoch {epoch + 1} | Train Loss: {avg_train_loss:.4f} | Val WER: {val_wer:.4f} | Val TER: {val_ter:.4f}")




epoch,▁▅█
train/loss,█▃▄▄▂▂▁▂▄▃▂▃▃▃▁▃▅▃▅▃▄▁▂▂▂▆▂▁▃▃▂▂▄▃▂▄▂▂▂▁
train/loss_avg,█▃▁
val/ter,▁▅█
val/wer,▄█▁
epoch,3
train/loss,0.59491
train/loss_avg,0.08697
val/ter,0.52372
val/wer,0.85078


Evaluating: 100%|██████████| 155/155 [00:41<00:00,  3.76it/s]


Epoch 1 | Train Loss: -0.1748 | Val WER: 0.5441 | Val TER: 0.0575


Evaluating: 100%|██████████| 155/155 [00:41<00:00,  3.74it/s]


Epoch 2 | Train Loss: -0.2193 | Val WER: 0.5133 | Val TER: 0.0579


Evaluating: 100%|██████████| 155/155 [00:41<00:00,  3.76it/s]


Epoch 3 | Train Loss: -0.2848 | Val WER: 0.5027 | Val TER: 0.0520
